In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.utils.data as Data
import torch.optim as optim

In [2]:
sentences = ["i like dog", "i love coffee", "i hate milk"]
word_list = ' '.join(sentences).split()
vocab = list(set(word_list))
vocab_size = len(vocab)
word2idx = {w:i for i,w in enumerate(vocab)}
idx2word = {i:w for i,w in enumerate(vocab)}

In [3]:
def make_data(sentences):
    inputs=[]
    outputs=[]
    for sentence in sentences:
        words = sentence.split()
        idxs = [word2idx[word] for word in words[:-1]]
        inputs.append([np.eye(vocab_size)[idx] for idx in idxs])
        outputs.append(word2idx[words[-1]])
    return inputs,outputs

In [4]:
# TextRNN Parameter
batch_size = 2
n_step = 2 # number of cells(= number of Step)
n_hidden = 5 # number of hidden units in one cell

In [5]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN,self).__init__()
        self.rnn = nn.RNN(input_size=vocab_size,hidden_size=n_hidden,num_layers=n_step,batch_first=True)
        self.fc = nn.Linear(n_hidden,vocab_size)
    def forward(self,X,hidden):
        output, hn = self.rnn(X,hidden)
        return self.fc(output[:,-1,:])

In [6]:
inputs,outputs = make_data(sentences)
inputs,outputs = torch.FloatTensor(inputs),torch.LongTensor(outputs)
dataset = Data.TensorDataset(inputs,outputs)
loader = Data.DataLoader(dataset,batch_size,True)

In [7]:
model = TextRNN().cuda()
optimizer = optim.Adam(model.parameters(),lr=1e-3)
criterion = nn.CrossEntropyLoss()



In [8]:
for epoch in range(5000):
    for x,y in loader:
        hidden = torch.zeros(n_step,x.shape[0],  n_hidden) # h0
        pred = model(x.cuda(),hidden.cuda())
        loss = criterion(pred,y.cuda())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (epoch+1) %1000 ==0:
           print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss),end='\r\n')

Epoch: 1000 cost = 0.034593
Epoch: 1000 cost = 0.057116
Epoch: 2000 cost = 0.008908
Epoch: 2000 cost = 0.007036
Epoch: 3000 cost = 0.002360
Epoch: 3000 cost = 0.003078
Epoch: 4000 cost = 0.000920
Epoch: 4000 cost = 0.000913
Epoch: 5000 cost = 0.000365
Epoch: 5000 cost = 0.000281


In [18]:
model.eval()
input = [sen.split()[:2] for sen in sentences]
# Predict
hidden = torch.zeros(n_step, len(input), n_hidden)
predict = model(inputs.cuda(),hidden.cuda() ).cpu().data.max(1, keepdim=True)[1]

print([sen.split()[:2] for sen in sentences], '->', [idx2word[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
